In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "evaluate" "sentencepiece"

In [ ]:
!nvidia-smi

In [ ]:
from datasets import load_dataset, Dataset
from random import randrange
from tqdm import tqdm

In [ ]:
dt = load_dataset("lucasmccabe-lmi/CodeAlpaca-20k")

In [ ]:
dt["train"]

In [ ]:
 = n-2
    for i in range(n-1, -1, -1):
        if a[i] != 0:
            while j > 0 and a[j] != 0:
                j -= 1
            
            if a[j] == 0:
                c, b = a[j], a[i]
                a[j], a[i] = b, c
                j -= 1
            print(a)
    return adt["train"][0]

In [ ]:
dt = dt["train"].train_test_split(test_size=0.2, seed=7)

In [ ]:
dt

In [ ]:
train_data = dt["train"]
valid_data = dt["test"]

In [ ]:
def formatting_prompts_func(example):
    text = f"### Question: {example['instruction']}\n ### Answer: {example['output']}"
    return text

def chars_token_ratio(dataset, tokenizer, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = formatting_prompts_func(example)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    return total_characters / total_tokens

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from trl.trainer import ConstantLengthDataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
import numpy as np


# import evaluate
# metric = evaluate.load("bleu")


In [ ]:
model_id = "drive/MyDrive/flacon3b-int4-coding/merged_model/"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
seq_length = 1024
chars_per_token = chars_token_ratio(train_data, tokenizer)

In [ ]:
train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        formatting_func=formatting_prompts_func,
        infinite=True,
        seq_length=seq_length,
        chars_per_token=chars_per_token,
    )

valid_dataset = ConstantLengthDataset(
    tokenizer,
    valid_data,
    formatting_func=formatting_prompts_func,
    infinite=False,
    seq_length=seq_length,
    chars_per_token=chars_per_token,
)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
base_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache=False, device_map="auto")
base_model.config.pretraining_tp = 1
base_model.config.use_cache = False

In [ ]:
# adjust r based on your budget and requirement, higher r if you can afford to train more params

peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.1,
        r=2,
        target_modules=["q_proj","v_proj"], #"k_proj" ,
        bias="none",
        task_type="CAUSAL_LM",
)

# optinally enable gradient checkpointing if model is huge > 13b for 48gb vram gpu
# or number of params to train are high
# model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(base_model)
model = get_peft_model(base_model, peft_config)

In [ ]:
print_trainable_parameters(model)

In [ ]:
print(model)

In [ ]:
args = TrainingArguments(
    output_dir="drive/MyDrive/flacon3b-int4-coding",
    num_train_epochs=1,
    per_device_train_batch_size=6,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=2,
    save_strategy="steps",
    eval_steps=10,
    learning_rate=2e-4,
    bf16=True,
    tf32=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
)

In [ ]:
# def compute_metrics(eval_preds):
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     # Replace -100 in the labels as we can't decode them.
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#     result = {k: round(v * 100, 4) for k, v in result.items()}
#     prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
#     result["gen_len"] = np.mean(prediction_lens)
#     return result

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    # compute_metrics=compute_metrics,
    peft_config=peft_config,
    max_seq_length=seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=formatting_prompts_func,
    args=args,
)

In [ ]:
trainer.train()

In [ ]:
# trainer.evaluate()
# you can evaluate only if model is small and you have enough vram available

In [ ]:
output_dir = args.output_dir
trainer.model.save_pretrained(output_dir)

In [ ]:
# Free memory for merging weights
del base_model
torch.cuda.empty_cache()

In [ ]:
from peft import AutoPeftModelForCausalLM
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

output_merged_dir = f"{output_dir}/merged_model/"
model.save_pretrained(output_merged_dir, safe_serialization=True)